In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

In [ ]:
findspark.init("/home/ubuntu/software/spark")

In [8]:
spark = SparkSession.builder\
    .master("local").appName("music_recommendation_system").getOrCreate()

In [ ]:
final_dataframe = spark.createDataFrame([], StructType([]))

In [ ]:
for i in range(1,1000):
    data = "hdfs://54.201.83.165:9000/raw_data/mpd.slice." + str(i)
    temp_dataframe = spark.read.json(data)
    final_dataframe.append(temp_dataframe)

In [ ]:
final_dataframe.printSchema()

In [ ]:
playlist_df = final_dataframe.select('playlists', explode('playlists')).\
                    select(explode('col.tracks'),'col.name','col.pid',
                       'col.num_followers', 'col.num_tracks','col.duration_ms','col.num_artists').\
                        withColumnRenamed('col.num_followers', 'num_followers').\
                        withColumnRenamed('col.num_tracks', 'tracks').\
                        withColumnRenamed('col.duration_ms', 'duration_ms').\
                        withColumnRenamed('col.num_artists', 'num_artists').\
                        withColumnRenamed('col.pid', 'pid').\
                        withColumnRenamed('col.name', 'name').\
                        select('col.*','*').drop('col')

In [ ]:
# checking the non null count for tracks
playlist_df.where(playlist_df['playlists'].isNotNull()).select('tracks').distinct().count()

In [ ]:
# checking the null count for lyrics
playlist_df.where(playlist_df['playlists'].isNull()).select('tracks').distinct().count()

In [ ]:
split_col_values = split(playlist_df["tracks"], ‘\t’)
df_artist = playlist_df.withColumn("artist_id", split_col_values.getItem(0))
df_artist = playlist_df.withColumn("artist_name", split_col_values.getItem(1))
df_artist=df_artist.drop("tracks")
df_user_artist = playlist_df.withColumn('artist_id', split_col_values.getItem(0))
df_user_artist = playlist_df.withColumn('duration_ms', split_col_values.getItem(2))
df_user_artist=df.drop('tracks')
df_user_artist.show()

In [ ]:
train_df, test_df = df_user_artist.randomSplit([0.7,0.3],seed=7)

In [ ]:
als=ALS(maxIter = 10, regParam = 0.02, coldStartStrategy = “drop”,  implicitPrefs = True, userCol="artist_name", itemCol="artist_id", ratingCol="duration_ms")
model=als.fit(train_df)
predictions=model.trasnform(train_df)

In [ ]:
recom = model.recommendForAllUsers(20).show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName="rmse", labelCol="duration_ms", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(rmse)

In [ ]:
database = '*****'
collection = '*****'
user_name = '*****'
password = '*****'
address = '*****.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [ ]:
recom.write.format("mongo").option("uri",connection_string).mode("append").save()